In [1]:
import pandas as pd

### 1. Daten einlesen

In [4]:
# Bevölkerungsdaten laden
population_data = pd.read_csv(r'C:\Users\Admin\Desktop\EGM_2025\data\Cleaned\GeschlechtAlterQuartiere.csv')

print(f"Datensatz geladen: {len(population_data):,} Zeilen")
print(f"Verfügbare Jahre: {population_data['Year'].min()} - {population_data['Year'].max()}")
print(f"Verfügbare Kreise: {sorted(population_data['Kreis'].unique())}")

# Überblick über verfügbare Altersgruppen
print("\nVerfügbare Altersgruppen:")
age_groups = sorted(population_data['AgeGroup'].unique())
print(age_groups)

Datensatz geladen: 31,025 Zeilen
Verfügbare Jahre: 1993 - 2024
Verfügbare Kreise: [np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9), np.int64(10), np.int64(11), np.int64(12)]

Verfügbare Altersgruppen:
['0-4', '10-14', '100 u. älter', '15-19', '20-24', '25-29', '30-34', '35-39', '40-44', '45-49', '5-9', '50-54', '55-59', '60-64', '65-69', '70-74', '75-79', '80-84', '85-89', '90-94', '95-99']


### 2. Annahmen definieren

In [5]:
# Unsere konservativen Annahmen
assumptions = {
    # Überlebensraten (10 Jahre)
    'survival_55_59': 0.90,    # 90% der 55-59 Jährigen erreichen 65+
    'survival_60_64': 0.95,    # 95% der 60-64 Jährigen erreichen 70+
    'survival_65_69': 0.90,    # 90% der 65-69 Jährigen erreichen 75+
    'survival_70_74': 0.85,    # 85% der 70-74 Jährigen erreichen 80+
    'survival_75_79': 0.70,    # 70% der 75-79 Jährigen erreichen 85+
    'survival_80_84': 0.50,    # 50% der 80-84 Jährigen erreichen 90+
    
    # Retention Raten (bleiben im gleichen Kreis)
    'retention_new_seniors': 0.85,     # 85% der neuen Senioren bleiben
    'retention_aging_seniors': 0.80,   # 80% der alternden Senioren bleiben
}

print("ÜBERLEBENSRATEN (10 Jahre):")
print(f"  55-59 → 65+: {assumptions['survival_55_59']*100:.0f}%")
print(f"  60-64 → 70+: {assumptions['survival_60_64']*100:.0f}%")
print(f"  65-69 → 75+: {assumptions['survival_65_69']*100:.0f}%")
print(f"  70-74 → 80+: {assumptions['survival_70_74']*100:.0f}%")
print(f"  75-79 → 85+: {assumptions['survival_75_79']*100:.0f}%")
print(f"  80-84 → 90+: {assumptions['survival_80_84']*100:.0f}%")

print("\nRETENTION RATEN (bleiben im Kreis):")
print(f"  Neue Senioren: {assumptions['retention_new_seniors']*100:.0f}%")
print(f"  Alternde Senioren: {assumptions['retention_aging_seniors']*100:.0f}%")

ÜBERLEBENSRATEN (10 Jahre):
  55-59 → 65+: 90%
  60-64 → 70+: 95%
  65-69 → 75+: 90%
  70-74 → 80+: 85%
  75-79 → 85+: 70%
  80-84 → 90+: 50%

RETENTION RATEN (bleiben im Kreis):
  Neue Senioren: 85%
  Alternde Senioren: 80%


### 3. Daten für 2024 filtern und vorbereiten

In [6]:
# Nur 2024 Daten
data_2024 = population_data[population_data['Year'] == 2024].copy()

# Relevante Altersgruppen filtern
target_age_groups = ['55-59', '60-64', '65-69', '70-74', '75-79', '80-84']
data_2024_filtered = data_2024[data_2024['AgeGroup'].isin(target_age_groups)].copy()

print(f"Rohdatensätze für 2024: {len(data_2024_filtered):,}")
print(f"Analysierte Altersgruppen: {target_age_groups}")

# WICHTIG: Nach Kreis und Altersgruppe aggregieren (Sex und Origin zusammenfassen!)
print("\nAggregiere Daten nach Kreis und Altersgruppe...")
print("(kombiniert alle Sex- und Origin-Kategorien pro Kreis/Altersgruppe)")

kreis_age_2024 = data_2024_filtered.groupby(['Kreis', 'AgeGroup'])['PopulationCount'].sum().reset_index()

print(f"\nAggregierte Daten: {len(kreis_age_2024)} Kreis-Altersgruppen-Kombinationen")

# Beispiel zeigen: Wie die Aggregation funktioniert
print("\nBeispiel Aggregation für Kreis 1, Altersgruppe 55-59:")
example_kreis1 = data_2024_filtered[(data_2024_filtered['Kreis'] == 1) & 
                                   (data_2024_filtered['AgeGroup'] == '55-59')]
if len(example_kreis1) > 0:
    print("Vor Aggregation (alle Kategorien):")
    print(example_kreis1[['Sex', 'Origin', 'PopulationCount']])
    total_kreis1 = example_kreis1['PopulationCount'].sum()
    print(f"Nach Aggregation: {total_kreis1} Personen total")

# Pivot-Tabelle für bessere Übersicht
population_matrix_2024 = kreis_age_2024.pivot(index='Kreis', columns='AgeGroup', values='PopulationCount').fillna(0)

print("\nPopulation Matrix 2024 (erste 5 Kreise):")
print(population_matrix_2024.head())

Rohdatensätze für 2024: 288
Analysierte Altersgruppen: ['55-59', '60-64', '65-69', '70-74', '75-79', '80-84']

Aggregiere Daten nach Kreis und Altersgruppe...
(kombiniert alle Sex- und Origin-Kategorien pro Kreis/Altersgruppe)

Aggregierte Daten: 72 Kreis-Altersgruppen-Kombinationen

Beispiel Aggregation für Kreis 1, Altersgruppe 55-59:
Vor Aggregation (alle Kategorien):
      Sex  Origin  PopulationCount
30598   M       1              167
30599   M       2               54
30600   W       1              126
30601   W       2               36
Nach Aggregation: 383 Personen total

Population Matrix 2024 (erste 5 Kreise):
AgeGroup  55-59  60-64  65-69  70-74  75-79  80-84
Kreis                                             
1           383    317    258    238    233    180
2          2088   1687   1299   1161   1035    877
3          2771   2251   1648   1364   1159    872
4          1685   1259    886    698    491    291
5           970    733    519    328    259    152


### 4. Baseline

In [7]:
# Aktuelle 65+ Population pro Kreis berechnen
current_65_plus_groups = ['65-69', '70-74', '75-79', '80-84']
current_65_plus = population_matrix_2024[current_65_plus_groups].sum(axis=1).reset_index()
current_65_plus.columns = ['Kreis', 'Population_65plus_2024']

# Sortieren nach Population
current_65_plus = current_65_plus.sort_values('Population_65plus_2024', ascending=False)

# Durchschnitt für Hotspot-Definition
avg_population_2024 = current_65_plus['Population_65plus_2024'].mean()

# Hotspots definieren (wie in der ursprünglichen Analyse)
current_65_plus['is_demographic_hotspot_2024'] = (current_65_plus['Population_65plus_2024'] > avg_population_2024).astype(int)

# Intensität definieren
def get_intensity_2024(population, avg):
    if population > avg * 1.5:
        return "high"
    elif population > avg:
        return "medium"
    else:
        return "low"

current_65_plus['hotspot_intensity_2024'] = current_65_plus.apply(
    lambda row: get_intensity_2024(row['Population_65plus_2024'], avg_population_2024),
    axis=1
)

print(f"Durchschnittliche 65+ Population 2024: {avg_population_2024:.0f}")
print(f"Anzahl demografischer Hotspots 2024: {current_65_plus['is_demographic_hotspot_2024'].sum()}")

print("\nAKTUELLE DEMOGRAFISCHE HOTSPOTS 2024:")
hotspots_2024 = current_65_plus[current_65_plus['is_demographic_hotspot_2024'] == 1]
print(hotspots_2024[['Kreis', 'Population_65plus_2024', 'hotspot_intensity_2024']])

Durchschnittliche 65+ Population 2024: 4288
Anzahl demografischer Hotspots 2024: 6

AKTUELLE DEMOGRAFISCHE HOTSPOTS 2024:
    Kreis  Population_65plus_2024 hotspot_intensity_2024
10     11                    8459                   high
8       9                    7180                   high
6       7                    6442                   high
9      10                    5313                 medium
2       3                    5043                 medium
1       2                    4372                 medium


### 5. Kohortenverschiebung durchführen

In [8]:
# Initialisiere Ergebnis-DataFrame
projection_2034 = population_matrix_2024.copy()
projection_2034.columns = [f"current_{col}" for col in projection_2034.columns]

# Neue Spalten für 2034 Projektionen
cohort_results = pd.DataFrame(index=projection_2034.index)

print("Berechne Kohortenverschiebung...")

# 5.1 Neue Senioren (55-59 → 65-69, 60-64 → 70-74)
print("\n5.1 Neue Senioren:")

# 55-59 werden zu 65-69
new_65_69 = (projection_2034['current_55-59'] * 
              assumptions['survival_55_59'] * 
              assumptions['retention_new_seniors'])

# 60-64 werden zu 70-74  
new_70_74 = (projection_2034['current_60-64'] * 
              assumptions['survival_60_64'] * 
              assumptions['retention_new_seniors'])

cohort_results['projected_65-69'] = new_65_69
cohort_results['projected_70-74'] = new_70_74

print(f"55-59 → 65-69: {new_65_69.sum():.0f} Personen (von {projection_2034['current_55-59'].sum():.0f})")
print(f"60-64 → 70-74: {new_70_74.sum():.0f} Personen (von {projection_2034['current_60-64'].sum():.0f})")

# 5.2 Alternde Senioren (65+ verschieben sich nach oben)
print("\n5.2 Alternde Senioren:")

# 65-69 werden zu 75-79
aging_75_79 = (projection_2034['current_65-69'] * 
                assumptions['survival_65_69'] * 
                assumptions['retention_aging_seniors'])

# 70-74 werden zu 80-84
aging_80_84 = (projection_2034['current_70-74'] * 
                assumptions['survival_70_74'] * 
                assumptions['retention_aging_seniors'])

# 75-79 werden zu 85-89 (über unseren Analysebereich hinaus)
aging_85_plus = (projection_2034['current_75-79'] * 
                  assumptions['survival_75_79'] * 
                  assumptions['retention_aging_seniors'])

# 80-84 werden zu 90-94 (über unseren Analysebereich hinaus)
aging_90_plus = (projection_2034['current_80-84'] * 
                  assumptions['survival_80_84'] * 
                  assumptions['retention_aging_seniors'])

cohort_results['projected_75-79'] = aging_75_79
cohort_results['projected_80-84'] = aging_80_84
cohort_results['projected_85+'] = aging_85_plus + aging_90_plus  # Kombiniert

print(f"65-69 → 75-79: {aging_75_79.sum():.0f} Personen")
print(f"70-74 → 80-84: {aging_80_84.sum():.0f} Personen")
print(f"75-79+ → 85+: {(aging_85_plus + aging_90_plus).sum():.0f} Personen")

Berechne Kohortenverschiebung...

5.1 Neue Senioren:
55-59 → 65-69: 19563 Personen (von 25573)
60-64 → 70-74: 17020 Personen (von 21077)

5.2 Alternde Senioren:
65-69 → 75-79: 11542 Personen
70-74 → 80-84: 9422 Personen
75-79+ → 85+: 10564 Personen


### 6. Gesamte 65+ Population 2034 berechnen

In [9]:
# Alle 65+ Gruppen für 2034 summieren
cohort_results['total_65plus_2034'] = (
    cohort_results['projected_65-69'] +
    cohort_results['projected_70-74'] +
    cohort_results['projected_75-79'] +
    cohort_results['projected_80-84'] +
    cohort_results['projected_85+']
)

# Mit aktuellen Werten vergleichen
comparison = pd.DataFrame({
    'Kreis': cohort_results.index,
    'Population_2024': current_65_plus.set_index('Kreis')['Population_65plus_2024'],
    'Population_2034': cohort_results['total_65plus_2034'],
})

comparison['Absolute_Change'] = comparison['Population_2034'] - comparison['Population_2024']
comparison['Percent_Change'] = ((comparison['Population_2034'] - comparison['Population_2024']) / 
                                comparison['Population_2024'] * 100)

comparison = comparison.sort_values('Population_2034', ascending=False)

print("VERGLEICH 2024 vs 2034:")
print(comparison.round(1))

VERGLEICH 2024 vs 2034:
       Kreis  Population_2024  Population_2034  Absolute_Change  \
Kreis                                                             
11        11             8459          11353.6           2894.6   
9          9             7180           9082.0           1902.0   
7          7             6442           7654.6           1212.6   
3          3             5043           7049.4           2006.4   
10        10             5313           6806.2           1493.2   
2          2             4372           5614.7           1242.7   
6          6             3978           5415.5           1437.5   
12        12             3825           5057.8           1232.8   
4          4             2366           3809.6           1443.6   
8          8             2317           3031.9            714.9   
5          5             1258           2136.5            878.5   
1          1              909           1099.1            190.1   

       Percent_Change  
Kreis        

### 7. Kohortenverschiebung: Was kristallisiert sich heraus?

In [10]:
# Durchschnitt für 2034 berechnen
avg_population_2034 = comparison['Population_2034'].mean()

# Hotspots für 2034 definieren (GLEICHE METHODIK wie 2024!)
comparison['is_demographic_hotspot_2034'] = (comparison['Population_2034'] > avg_population_2034).astype(int)

# Intensität für 2034 definieren (GLEICHE METHODIK wie 2024!)
def get_intensity_2034(population, avg):
    if population > avg * 1.5:
        return "high"
    elif population > avg:
        return "medium"
    else:
        return "low"

comparison['hotspot_intensity_2034'] = comparison.apply(
    lambda row: get_intensity_2034(row['Population_2034'], avg_population_2034),
    axis=1
)

print(f"Durchschnitt 2024: {avg_population_2024:.0f} Personen")
print(f"Durchschnitt 2034: {avg_population_2034:.0f} Personen")
print(f"Änderung Durchschnitt: {((avg_population_2034/avg_population_2024)-1)*100:+.1f}%")

print(f"\nAnzahl Hotspots 2024: {current_65_plus['is_demographic_hotspot_2024'].sum()}")
print(f"Anzahl Hotspots 2034: {comparison['is_demographic_hotspot_2034'].sum()}")

Durchschnitt 2024: 4288 Personen
Durchschnitt 2034: 5676 Personen
Änderung Durchschnitt: +32.4%

Anzahl Hotspots 2024: 6
Anzahl Hotspots 2034: 5


### 8. Der entscheidende Vergleich: 2024 vs 2034 Hotspots

In [14]:
# DEBUG: Schauen wir uns die DataFrames genau an
print("=== DEBUG INFO ===")
print("comparison DataFrame:")
print(f"Index: {comparison.index}")
print(f"Columns: {comparison.columns.tolist()}")
print(f"Shape: {comparison.shape}")
print(comparison.head())

print("\ncurrent_65_plus DataFrame:")
print(f"Index: {current_65_plus.index}")
print(f"Columns: {current_65_plus.columns.tolist()}")
print(f"Shape: {current_65_plus.shape}")
print(current_65_plus.head())

# LÖSUNG: Index komplett zurücksetzen für beide DataFrames
comparison = comparison.reset_index(drop=True)
current_65_plus = current_65_plus.reset_index(drop=True)

print("\nNach reset_index():")
print(f"comparison Index: {comparison.index}")
print(f"current_65_plus Index: {current_65_plus.index}")

# Jetzt sollte der Merge funktionieren
final_comparison = comparison.merge(
    current_65_plus[['Kreis', 'is_demographic_hotspot_2024', 'hotspot_intensity_2024']], 
    on='Kreis'
)

=== DEBUG INFO ===
comparison DataFrame:
Index: Index([11, 9, 7, 3, 10, 2, 6, 12, 4, 8, 5, 1], dtype='int64', name='Kreis')
Columns: ['Kreis', 'Population_2024', 'Population_2034', 'Absolute_Change', 'Percent_Change', 'is_demographic_hotspot_2034', 'hotspot_intensity_2034']
Shape: (12, 7)
       Kreis  Population_2024  Population_2034  Absolute_Change  \
Kreis                                                             
11        11             8459       11353.6250        2894.6250   
9          9             7180        9081.9800        1901.9800   
7          7             6442        7654.6350        1212.6350   
3          3             5043        7049.4175        2006.4175   
10        10             5313        6806.1525        1493.1525   

       Percent_Change  is_demographic_hotspot_2034 hotspot_intensity_2034  
Kreis                                                                      
11          34.219470                            1                   high  
9           

In [15]:
# DEBUG: Index-Problem beheben
print("Bereite DataFrames für Merge vor...")
comparison_clean = comparison.reset_index(drop=True)
current_65_plus_clean = current_65_plus.reset_index(drop=True)

print(f"Kreise in comparison: {sorted(comparison_clean['Kreis'].unique())}")
print(f"Kreise in current_65_plus: {sorted(current_65_plus_clean['Kreis'].unique())}")

# Merge der Hotspot-Informationen
final_comparison = comparison_clean.merge(
    current_65_plus_clean[['Kreis', 'is_demographic_hotspot_2024', 'hotspot_intensity_2024']], 
    on='Kreis',
    how='left'
)

print(f"Merge erfolgreich! Shape: {final_comparison.shape}")

# Hotspot-Status Kategorien
def categorize_hotspot_change(row):
    if row['is_demographic_hotspot_2024'] == 1 and row['is_demographic_hotspot_2034'] == 1:
        return "Stabil: Bleibt Hotspot"
    elif row['is_demographic_hotspot_2024'] == 0 and row['is_demographic_hotspot_2034'] == 1:
        return "Neu: Wird Hotspot"
    elif row['is_demographic_hotspot_2024'] == 1 and row['is_demographic_hotspot_2034'] == 0:
        return "Verlust: War Hotspot"
    else:
        return "Stabil: Kein Hotspot"

final_comparison['Hotspot_Change'] = final_comparison.apply(categorize_hotspot_change, axis=1)

# Sortieren für bessere Darstellung
final_comparison_sorted = final_comparison.sort_values('Population_2034', ascending=False)

print("\nVOLLSTÄNDIGER VERGLEICH (nach 2034 Population sortiert):")
print(final_comparison_sorted[['Kreis', 'Population_2024', 'Population_2034', 
                              'Percent_Change', 'hotspot_intensity_2024', 
                              'hotspot_intensity_2034', 'Hotspot_Change']].round(1))

Bereite DataFrames für Merge vor...
Kreise in comparison: [np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9), np.int64(10), np.int64(11), np.int64(12)]
Kreise in current_65_plus: [np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9), np.int64(10), np.int64(11), np.int64(12)]
Merge erfolgreich! Shape: (12, 9)

VOLLSTÄNDIGER VERGLEICH (nach 2034 Population sortiert):
    Kreis  Population_2024  Population_2034  Percent_Change  \
0      11             8459          11353.6            34.2   
1       9             7180           9082.0            26.5   
2       7             6442           7654.6            18.8   
3       3             5043           7049.4            39.8   
4      10             5313           6806.2            28.1   
5       2             4372           5614.7            28.4   
6       6             3978           5415.5            36.1   

### 9. was kommt dabei heraus? - Analyse

In [16]:
# Verschiedene Kategorien analysieren
stable_hotspots = final_comparison[final_comparison['Hotspot_Change'] == 'Stabil: Bleibt Hotspot']
new_hotspots = final_comparison[final_comparison['Hotspot_Change'] == 'Neu: Wird Hotspot']
lost_hotspots = final_comparison[final_comparison['Hotspot_Change'] == 'Verlust: War Hotspot']

print("A) STABILITÄT DER HOTSPOTS:")
if len(stable_hotspots) > 0:
    print("   Kreise, die STABIL Hotspots bleiben:")
    for _, row in stable_hotspots.iterrows():
        print(f"   → Kreis {row['Kreis']}: {row['hotspot_intensity_2024']} → {row['hotspot_intensity_2034']} ({row['Percent_Change']:+.1f}%)")
else:
    print("   KEINE stabilen Hotspots!")

print("\nB) NEUE HOTSPOTS:")
if len(new_hotspots) > 0:
    print("   Kreise, die zu NEUEN Hotspots werden:")
    for _, row in new_hotspots.iterrows():
        print(f"   → Kreis {row['Kreis']}: {row['hotspot_intensity_2024']} → {row['hotspot_intensity_2034']} ({row['Percent_Change']:+.1f}%)")
else:
    print("   KEINE neuen Hotspots!")

print("\nC) VERLORENE HOTSPOTS:")
if len(lost_hotspots) > 0:
    print("   Kreise, die Hotspot-Status VERLIEREN:")
    for _, row in lost_hotspots.iterrows():
        print(f"   → Kreis {row['Kreis']}: {row['hotspot_intensity_2024']} → {row['hotspot_intensity_2034']} ({row['Percent_Change']:+.1f}%)")
else:
    print("   KEINE verlorenen Hotspots!")

# Methodische Interpretation
print("\nD) METHODISCHE INTERPRETATION:")
total_hotspots_2024 = len(stable_hotspots) + len(lost_hotspots)
total_hotspots_2034 = len(stable_hotspots) + len(new_hotspots)
stability_rate = len(stable_hotspots) / total_hotspots_2024 * 100 if total_hotspots_2024 > 0 else 0

print(f"   Hotspot-Stabilität: {stability_rate:.1f}% ({len(stable_hotspots)}/{total_hotspots_2024})")
print(f"   Neue Hotspots: {len(new_hotspots)} Kreise")
print(f"   Verlorene Hotspots: {len(lost_hotspots)} Kreise")

if stability_rate >= 70:
    print("   → HOHE STABILITÄT: Unsere Hotspot-Methodik ist robust!")
elif len(stable_hotspots) == total_hotspots_2024 and len(new_hotspots) == 0:
    print("   → PERFEKTE STABILITÄT: Exakt dieselben Hotspots!")
else:
    print("   → DYNAMISCHE VERÄNDERUNG: Demografischer Wandel verändert Hotspots!")

A) STABILITÄT DER HOTSPOTS:
   Kreise, die STABIL Hotspots bleiben:
   → Kreis 11: high → high (+34.2%)
   → Kreis 9: high → high (+26.5%)
   → Kreis 7: high → medium (+18.8%)
   → Kreis 3: medium → medium (+39.8%)
   → Kreis 10: medium → medium (+28.1%)

B) NEUE HOTSPOTS:
   KEINE neuen Hotspots!

C) VERLORENE HOTSPOTS:
   Kreise, die Hotspot-Status VERLIEREN:
   → Kreis 2: medium → low (+28.4%)

D) METHODISCHE INTERPRETATION:
   Hotspot-Stabilität: 83.3% (5/6)
   Neue Hotspots: 0 Kreise
   Verlorene Hotspots: 1 Kreise
   → HOHE STABILITÄT: Unsere Hotspot-Methodik ist robust!


In [17]:
# Final DataFrame für Export
export_df = final_comparison_sorted[['Kreis', 'Population_2024', 'Population_2034', 
                                    'Absolute_Change', 'Percent_Change',
                                    'is_demographic_hotspot_2024', 'hotspot_intensity_2024',
                                    'is_demographic_hotspot_2034', 'hotspot_intensity_2034',
                                    'Hotspot_Change']].round(1)

export_df.to_csv('cohort_analysis_2024_vs_2034.csv', index=False)

print("Ergebnisse gespeichert als: 'cohort_analysis_2024_vs_2034.csv'")
print(f"Exportiert: {len(export_df)} Kreise mit vollständiger Kohortenanalyse")

Ergebnisse gespeichert als: 'cohort_analysis_2024_vs_2034.csv'
Exportiert: 12 Kreise mit vollständiger Kohortenanalyse
